In [0]:
# PROJECT_PATH = '/home/dobos/project/pfs_targeting/python:' +\
#     '/home/dobos/project/pfsspec-all:' +\
#     '/home/dobos/project/pfs_cmdfit/python:' +\
#     '/home/dobos/project/pfs_isochrones/python:' +\
#     '/home/dobos/project/ics_cobraOps/python:' +\
#     '/home/dobos/project/ics_cobraCharmer/python:' +\
#     '/home/dobos/project/ets_fiberalloc:' +\
#     '/home/dobos/project/pfs_utils/python:' +\
#     '../pysynphot:' +\
#     '../SciScript-Python/py3'

ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc_nb_bosz'
OBS_PATH = '/datascope/subaru/data/cmdfit/dSph/umi_tpall3e_g24.cat'
SIM_PATH = '/datascope/subaru/data/cmdfit/run/umi/sim/mix_bin_200k_hsc_007/sample.h5'
MEMBERS_PATH = '/datascope/subaru/data/cmdfit/dSph/spectro_members_umi.csv'

In [0]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle

# Plot style

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Configurations

In [0]:
configs = {}

for nb in [True, False]:
    for targeting in ['brightest', 'faintest', 'maxp']:
        c = {
            'nb': nb,
            'targeting': targeting
        }
        configs[targeting + ('' if not nb else '_nb')] = c

configs

# Photometry

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
hsc = SubaruHSC.photometry()
cmd = CMD([
        ColorAxis(
            Color([hsc.magnitudes['g'], hsc.magnitudes['i']]),
            limits=(-1, 4)),
        MagnitudeAxis(
            hsc.magnitudes['g'],
            limits=(15.5, 24.5)
        )
    ])
ccd = CCD([
    cmd.axes[0],
    ColorAxis(
        Color([hsc.magnitudes['g'], hsc.magnitudes['nb515']]),
        limits=(-0.5, 0.5)
    )
])

# Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

# Load observations

In [0]:
from pfs.ga.targeting.io import TextObservationReader

In [0]:
#obs = TextObservationReader().read_hsc(OBS_PATH)
obs = SubaruHSC.text_observation_reader().read(OBS_PATH)
obs.data.head()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs)
ccd.plot_observation(axs[1], obs)

f.tight_layout()

In [0]:
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV

In [0]:
ra, dec = obs.get_coords()
print(ra.mean(), dec.mean())

In [0]:
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
print(pointing.ra, pointing.dec)
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120, subplot_kw=dict(projection=wcs.wcs))
fov.plot_catalog(ax, obs)

## Load spectroscopically confirmed members

In [0]:
from pfs.ga.targeting.data import Observation

In [0]:
df = pd.read_csv(MEMBERS_PATH, skiprows=1,
                 names=['id', 'RA', 'Dec', 'mem', 'ids', 'cat', 'matched_ids'])

df['mem'] = df['mem'].map(lambda x: x > 0.5)
df['ids'] = df['ids'].astype(int)
df['matched_ids'] = df['matched_ids'].astype(int)

df[df['mem']]

In [0]:
mem = Observation()
mem._set_data(df[df['mem']])

mem.data

In [0]:
idx, sep = obs.cross_match(mem)
idx.shape, sep.shape, sep.min(), sep.max()

In [0]:
confirmed_mask = (sep < 0.5 / 3600.)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, c='grey')
cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=3)

ccd.plot_observation(axs[1], obs, c='grey')
ccd.plot_observation(axs[1], obs, mask=confirmed_mask, c='r', size=3)

f.suptitle('Confirmed members')
f.tight_layout()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120, subplot_kw=dict(projection=wcs.wcs))

fov.plot_catalog(ax, obs, c='grey')
fov.plot_catalog(ax, obs, mask=confirmed_mask, c='r', size=3)

f.suptitle('Confirmed members')
f.tight_layout()

# Load simulation

In [0]:
from pfs.ga.targeting.io import Hdf5SimulationReader

In [0]:
r = Hdf5SimulationReader()
cm = {}
for prefix in ['', 'obs_', 'err_', 'flux_', 'obs_flux_', 'err_flux_', 'counts_', 'obs_counts_', 'err_counts_']:
    cm[prefix + 'hsc_g2'] = prefix + 'hsc_g'
    cm[prefix + 'hsc_i2'] = prefix + 'hsc_i'
r.column_mapping = cm
r.append_photometry(SubaruHSC.photometry())
sim = r.read(SIM_PATH)

In [0]:
for k in sim.data.keys():
    print(k, sim.data[k].shape)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_simulation(axs[0], sim, s=np.s_[::100], size=0.05)
ccd.plot_simulation(axs[1], sim, s=np.s_[::100], size=0.05)

f.tight_layout()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_simulation(axs[0], sim, apply_categories=True, s=np.s_[::20], size=0.05)
ccd.plot_simulation(axs[1], sim, apply_categories=True, s=np.s_[::20], size=0.05)

f.tight_layout()

# Define selections

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling, MagnitudeSelection, ColorSelection, LinearSelection

In [0]:
def get_selection_mask(catalog, nb=True, blue=False, probcut=None, observed=None, bright=16, faint=23.5):
    
    # Broadband colors
    mask = ColorSelection(ccd.axes[0], 0.12, 2.0).apply(catalog, observed=observed)

    # Narrow band
    if nb:
        mask &= (
            ColorSelection(ccd.axes[0], 0.12, 0.5).apply(catalog, observed=observed)

            | ColorSelection(ccd.axes[1], 0.1, None).apply(catalog, observed=observed)
            & ColorSelection(ccd.axes[0], None, 1.65).apply(catalog, observed=observed)
            
            | LinearSelection(ccd.axes, [-0.25, 1.0], -0.15, None).apply(catalog, observed=observed)
        )

    # Probability-based cut (map) - nonzero membership probability
    if probcut is not None:
        mask &= probcut.apply(catalog, observed=observed)

    # Allow blue
    if blue:
        mask |= (
            ColorSelection(ccd.axes[0], None, 0.12).apply(catalog, observed=observed)
        )

    # Always impose faint and bright magnitude cuts
    mask &= MagnitudeSelection(cmd.axes[1], bright, faint).apply(catalog, observed=observed)

    return mask

In [0]:
f, ax = plt.subplots(1, 1)

mask = get_selection_mask(obs, observed=True, nb=True, blue=True)
cmd.plot_catalog(ax, obs, observed=True, mask=mask)

# Compare simulation to observations

In [0]:
# Original weights
print('data.w', sim.data['w'])

w = np.bincount(sim.data['g']) / sim.data['g'].shape
print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

# New weights, boost thick disk and halo
w1 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
w1[2:4] *= 100
# w1[0:6] *= 3  # thin disk 1-3
# w1[4:6] *= 1.2  # thin disk 3
w1[6:8] *= 15  # thick disk
w1[8:10] *= 28   # halo
##### good for histograms w1[10:12] *= 18  # dSph
##### good for ghost plot
w1[10:12] *= 50  # dSph
w1 /= w1.sum()
print('w1', w1, np.sum(w1[:-2]), np.sum(w1[-2:]))

# New categories
g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
print('g1', g1.shape)

# Verify new categories
w2 = np.bincount(g1) / g1.shape
print('w2', w2, np.sum(w2[:-2]), np.sum(w2[-2:]))

In [0]:
# Calculate the best combination of weight by fitting the color histogram of observations

In [0]:
sim.data['g'], g1

In [0]:
# Number of objects inside cuts
mask = get_selection_mask(obs, nb=False, observed=True)
n_obs = mask.sum()
print('obs', n_obs)

#(x, x_err), (y, y_err) = obs.get_diagram_values(cmd.axes, observed=True, mask=mask)
#mask = (0.1 < x) & (x < 2) & (y < 23.5)
#print('obs', n_obs)

mask = get_selection_mask(sim, nb=False, observed=True)
mask = sim.apply_categories(mask, g=g1)
n_sim = mask.sum()
print('sim', n_sim)

# (x, x_err), (y, y_err) = sim.get_diagram_values(cmd.axes, observed=True)
# x = sim.apply_categories(x, g=g1)
# y = sim.apply_categories(y, g=g1)
# mask = (0.1 < x) & (x < 2) & (y < 23.5)
# n_sim = mask.sum()
# print('sim', n_sim)

n_sim / n_obs

In [0]:
NB = False

f, axs = plt.subplots(1, 3, figsize=(6, 4), dpi=120)

mask = get_selection_mask(obs, nb=NB, observed=True)
cmd.plot_observation(axs[0], obs, size=0.05, mask=mask)

mask = get_selection_mask(sim, nb=NB, observed=True)
mask = sim.apply_categories(mask, g=g1)
cmd.plot_simulation(axs[1], sim, observed=True, apply_categories=True, mask=mask, g=g1, s=np.s_[::30], size=0.05)

mask = get_selection_mask(sim, nb=NB, observed=True)
mask = sim.apply_categories(mask, g=sim.data['g'])
cmd.plot_simulation(axs[2], sim, observed=True, apply_categories=True, mask=mask, g=sim.data['g'], s=np.s_[::30], size=0.05)

for ax in axs:
    ax.grid()
    ax.set_xlim(-0.2, 2.2)

f.tight_layout()

In [0]:
def plot_histogram(ax, obs, sim, axis, plot_populations=True):
    ((x, x_err),) = obs.get_diagram_values([axis], observed=True)
    mask = get_selection_mask(obs, nb=NB, observed=True)
    hist, bins = np.histogram(x[mask], bins=100, density=True)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='OBS')
    print(x.min(), x.max())

    ((x, x_err),) = sim.get_diagram_values([axis], observed=True)
    mask = get_selection_mask(sim, nb=NB, observed=True)
    mask = sim.apply_categories(mask, g=g1)
    x = sim.apply_categories(x, g=g1)
    hist, bins = np.histogram(x[mask], bins=100, density=True)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='SIM')
    
    if plot_populations:
        for i, name in zip(range(6), ['thin1', 'thin2', 'thin3', 'thick', 'halo', 'dSph']):
            hist, bins = np.histogram(x[mask][(g1[mask[:,0]] == 2 * i) | (g1[mask[:,0]] == 2 * i + 1)], bins=100, density=True)
            ax.step(0.5 * (bins[1:] + bins[:-1]), (w1[2 * i] + w1[2 * i + 1]) * hist, lw=1, label=name)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, cmd.axes[0])

ax.set_xlim(-0.2, 2.2)
ax.set_xlabel('HSC $g - i$')
ax.legend()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, cmd.axes[1])

ax.set_xlabel('HSC $g$')
ax.legend()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2.4), dpi=240)

plot_histogram(ax, obs, sim, MagnitudeAxis(hsc.magnitudes['i']))

ax.set_xlabel('HSC $i$')
ax.legend()

# Create the probability map

We create a probability map without properly normalizing it. What we need at this point is the outline of the valid are which is occupied by the member stars because it will be used for the color selection

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True, faint=23.5)
mask.shape, mask.sum()

In [0]:
w1

In [0]:
pmap = ProbabilityMap(cmd.axes)
pmap.from_simulation(sim, bins=[100, 100], extents=[[0.1, 2.0], [17.0, 23.5]],
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w1, observed=True, mask=mask)
pmap.maximum_filter()

In [0]:
pmap.extents

In [0]:
probcut = ProbabilityCut(pmap, 1, np.log(0.0001))

In [0]:
psamp = ProbabilitySampling(pmap, 0)
psamp_mask = psamp.apply(obs, observed=True)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
l1 = cmd.plot_probability_map(axs[1], pmap, 1)

f.tight_layout()

In [0]:
l0.get_clim(), l1.get_clim()

# Isochrones

In [0]:
from pfs.ga.targeting import Isochrone

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3, 2), dpi=240)

hist, bins = np.histogram(sim.data['Z'][:, -2], density=True, bins=100)
ax.plot(0.5 * (bins[1:] + bins[:-1]), hist)
ax.set_xlabel('[Fe/H]')

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3, 2), dpi=240)

hist, bins = np.histogram(sim.data['t'][:, -2], density=True, bins=100)
#ax.plot(10**(0.5 * (bins[1:] + bins[:-1]) - 9), hist)
#ax.set_xlabel('age [Gyr]')
ax.plot(0.5 * (bins[1:] + bins[:-1]) - 9, hist)
ax.set_xlabel('log age')

In [0]:
iso_blue = Isochrone()
iso_blue.from_isogrid(hsc, iso, Fe_H=-2.5, log_t=10.135, DM=19.2)

iso_red = Isochrone()
#iso_red.from_isogrid(hsc, iso, Fe_H=-1.2, log_t=10.11, DM=19.2)
iso_red.from_isogrid(hsc, iso, Fe_H=-1.6, log_t=10.11, DM=19.2)

In [0]:
iso_blue.get_diagram_values(cmd.axes)

# Whitepaper plot

In [0]:
# Number of points within rectangular selection
mask = get_selection_mask(obs, nb=False, probcut=None, observed=True)
print('obs:', mask.shape, mask.sum())

mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)
mask = sim.apply_categories(mask)
print('sim:', mask.shape, mask.sum(axis=0))

In [0]:
# Population weights without NB cuts
print('w1', w1, w1.sum())

# New categories
g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
print('g1', g1.shape)

# Verify new categories
w2 = np.bincount(g1) / g1.shape
print('w2', w2, w2.sum(), np.sum(w2[:-2]), np.sum(w2[-2:]))

# Population weights after cuts
mask = get_selection_mask(sim, nb=True, probcut=None, observed=True)
sim.apply_categories(mask, g=g1)
w3 = np.bincount(g1[mask[:,0]]) / mask.sum()
w3 /= w3.sum()
print('w3', w3, np.sum(w3), np.sum(w3[:-2]), np.sum(w3[-2:]))

# Population weights for the ghost plot
w = np.bincount(sim.data['g']) / sim.data['g'].shape
print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

# New weights, boost thick disk and halo
w4 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
w4[2:4] *= 100
# w4[0:6] *= 3  # thin disk 1-3
# w4[4:6] *= 1.2  # thin disk 3
w4[6:8] *= 15  # thick disk
w4[8:10] *= 28   # halo
w4[10:12] *= 50  # dSph
w4 /= w4.sum()
print('w4', w4, np.sum(w4[:-2]), np.sum(w4[-2:]))

In [0]:
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)

pmap_nonb = ProbabilityMap(cmd.axes)
pmap_nonb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w1, observed=True, mask=mask)
pmap_nonb.maximum_filter()

pmap_samp = ProbabilityMap(cmd.axes)
pmap_samp.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w4, observed=True, mask=mask)
pmap_samp.maximum_filter()

mask = get_selection_mask(sim, nb=True, probcut=None, observed=True)

pmap_nb = ProbabilityMap(cmd.axes)
pmap_nb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w3, observed=True, mask=mask)
pmap_nb.maximum_filter()

In [0]:
psamp = ProbabilitySampling(pmap_samp, 0)
psamp_mask = psamp.apply(obs, observed=True)

In [0]:
def hex_to_rgb(hex):
    return tuple(int(hex[i + 1:i + 3], 16) / 255 for i in (0, 2, 4))

def chop_cmap_frac(cmap: LinearSegmentedColormap, frac: float) -> LinearSegmentedColormap:
    """Chops off the beginning `frac` fraction of a colormap."""
    cmap_as_array = cmap(np.arange(256))
    cmap_as_array = cmap_as_array[:int(frac * len(cmap_as_array))]
    return LinearSegmentedColormap.from_list(cmap.name + f"_frac{frac}", cmap_as_array)

def get_nice_cmap(from_black=False):
    if from_black:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, hex_to_rgb('#595959')),
            (0.33, hex_to_rgb('#5F9ED1')),
            (0.66, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad((1.0, 1.0, 1.0))
    else:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, (1.0, 1.0, 1.0)),
            (0.15, hex_to_rgb('#ABABAB')),
            (0.25, hex_to_rgb('#595959')),
            (0.50, hex_to_rgb('#5F9ED1')),
            (0.75, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad(cmap(0))

    return cmap

#cmap = mpl.cm.get_cmap('terrain_r')

# cmap = LinearSegmentedColormap.from_list('mycmap',
#         [(0.00, (1.0, 1.0, 1.0)),
#          #(0.15, (0.75, 0.75, 0.75)),
#          (0.25, (1.0, 1.0, 0.6)),
#          (0.50, (0.0, 0.8, 0.4)),
#          (0.75, (0.0, 0.6, 1.0)),
#          (1.00, (1.0, 0.0, 0.0))])

#006BA4, #FF800E, #ABABAB, #595959,
#5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF

# cmap = mpl.cm.get_cmap('cubehelix_r')
# cmap = chop_cmap_frac(cmap, 0.5)

cmap = get_nice_cmap()

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap())

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap(from_black=True))

In [0]:
gs = GridSpec(ncols=6, nrows=1, width_ratios=[5, 5, 5, 5, 5, 0.3])
f = plt.figure(constrained_layout=False, figsize=(7.2, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(5) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
#cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

cmd.plot_observation(axs[2], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = cmd.plot_probability_map(axs[3], pmap_nonb, 1, cmap=cmap, rasterized=True)
l1 = cmd.plot_probability_map(axs[4], pmap_nb, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min', cmap=cmap)
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.55, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')

In [0]:
gs = GridSpec(ncols=3, nrows=1, width_ratios=[10, 10, 0.5])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

# cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
# #cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

# cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
# #cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
# #cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

# cmd.plot_observation(axs[0], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = cmd.plot_probability_map(axs[0], pmap_nonb, 1, cmap=cmap, rasterized=True)
l1 = cmd.plot_probability_map(axs[1], pmap_nb, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min')
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

for t in axc.get_yticklabels():
    t.set_horizontalalignment('right')   
    t.set_verticalalignment('center')   
    t.set_x(4.2)
    #t.set_y(-1)

f.tight_layout()

f.savefig('umi_hsc_pmap.pdf')

In [0]:
get_selection_mask(obs)

In [0]:
gs = GridSpec(ncols=2, nrows=1, width_ratios=[10, 10])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])

cmd.plot_observation(axs[0], obs, mask=psamp_mask, c='grey', rasterized=True)

psamp_outline_mask = get_selection_mask(obs, nb=True, probcut=None, observed=True)
cmd.plot_observation(axs[1], obs, mask=~psamp_mask & psamp_outline_mask, c='grey', rasterized=True)

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'ab'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel('HSC $g$')
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()

for ax in axs.flatten()[1:]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

f.tight_layout()

f.savefig('umi_hsc_ghost.pdf')

In [0]:
f, axs = plt.subplots(2, 2, figsize=(3.4, 4), dpi=240)

mask = get_selection_mask(obs, nb=True)

cmd.plot_observation(axs[0, 0], obs, c='grey', observed=True, rasterized=True)
cmd.plot_isochrone(axs[0, 0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c=hex_to_rgb('#006BA4'), lw=1)
cmd.plot_isochrone(axs[0, 0], iso_red, observed=True, error_sigma=[2.0, 2.0], c=hex_to_rgb('#FF800E'), lw=1)

cmd.plot_simulation(axs[0, 1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

ccd.plot_observation(axs[1, 0], obs, c='grey', observed=True, rasterized=True)
# ccd.plot_observation(axs[1, 0], obs, c='red', size=3, mask=mask, observed=True, rasterized=True)

ccd.plot_simulation(axs[1, 1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

for ax, pl in zip(axs.flatten(), 'abcd'):
    ax.text(0.1, 0.9, f'{pl})', fontsize=8, transform=ax.transAxes)
    ax.set_xlabel(r'HSC $g - i$')
    #ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    #ax.set_ylim(15.5, 24.5)
    #ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)

for ax in axs[0, :]:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax in axs[1, :]:
    ax.plot([0.12, 2.0, 2.0,
         1, 0.5, 0.5, 0.12, 0.12], 
        
        [0.5, 0.5, 0.1,
         0.1, -0.025, -0.5, -0.5, 0.5], 'k--', lw=1)

axs[0, 0].set_ylabel(r'HSC $g$')
axs[1, 0].set_ylabel(r'HSC $g - $NB$515$')

for ax in axs[:, 1]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

for ax in axs[0, :]:
    ax.set_xlabel(None)
    ax.set_xticklabels([])

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')

# Plot selection

In [0]:
def plot_selection_mask(catalog, mask, s=np.s_[:]):
    f, axs = plt.subplots(2, 2, figsize=(6, 6), dpi=120)

    cmd.plot_catalog(axs[0, 0], catalog, mask=~mask, s=s, c='k')
    cmd.plot_catalog(axs[0, 0], catalog, mask=mask, s=s, c='r')
    cmd.plot_catalog(axs[0, 1], catalog, mask=mask, s=s, c='k')

    ccd.plot_catalog(axs[1, 0], catalog, mask=~mask, s=s, c='k')
    ccd.plot_catalog(axs[1, 0], catalog, mask=mask, s=s, c='r')
    ccd.plot_catalog(axs[1, 1], catalog, mask=mask, s=s, c='k')

    f.tight_layout()

In [0]:
mask = get_selection_mask(obs, nb=False)
plot_selection_mask(obs, mask)

In [0]:
mask = get_selection_mask(sim, nb=False)
mask = sim.apply_categories(mask)
plot_selection_mask(sim, mask, s=np.s_[:40000])

In [0]:
mask = get_selection_mask(obs, nb=True)
plot_selection_mask(obs, mask)

In [0]:
mask = get_selection_mask(sim, nb=True)
mask = sim.apply_categories(mask)
plot_selection_mask(sim, mask, s=np.s_[:40000])

In [0]:
mask = get_selection_mask(obs, nb=True, probcut=probcut)
plot_selection_mask(obs, mask, s=np.s_[:40000])

In [0]:
mask = get_selection_mask(sim, nb=True, probcut=probcut)
mask = sim.apply_categories(mask)
plot_selection_mask(sim, mask, s=np.s_[:40000])

## Confirmed members after selection

In [0]:
mask = get_selection_mask(obs, nb=True, probcut=probcut)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_catalog(axs[0], obs, c='grey')
cmd.plot_catalog(axs[0], obs, mask=mask & confirmed_mask, c='r', size=3)

ccd.plot_catalog(axs[1], obs, c='grey')
ccd.plot_catalog(axs[1], obs, mask=mask & confirmed_mask, c='r', size=3)

f.suptitle('Confirmed members')
f.tight_layout()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120, subplot_kw=dict(projection=wcs.wcs))

fov.plot_catalog(ax, obs, c='grey')
fov.plot_catalog(ax, obs, mask=mask & confirmed_mask, c='r', size=3)

f.suptitle('Confirmed members')
f.tight_layout()

# Fit the radial profile

Here we fit to all stars **within the coarse color cuts** to have a better fit of the shape parameters. Later the foreground and the central density will be fitted again while keeping the morphological parameters constant. The initial foreground/member fraction will be determined from the radial profile fit.

In [0]:
from pfs.ga.targeting.projection import Pointing, Whitening
from pfs.ga.targeting.radialprofile import *

In [0]:
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
wcs = WcsProjection(pointing, proj='TAN')

fov = FOV(projection=wcs)

In [0]:
# Only the coarse color cuts
mask = get_selection_mask(obs, nb=False, probcut=None)

In [0]:
plot_selection_mask(obs, mask)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120,
    subplot_kw={'projection': fov.projection.wcs})

fov.plot_catalog(axs[0], obs)
fov.plot_catalog(axs[0], obs, mask=mask, color='r')

fov.plot_catalog(axs[1], obs, mask=mask)


In [0]:
w = Whitening(projection=wcs)
w.create(obs)

(x, y), _ = w.projection.world_to_pixel(obs.get_coords())

wxy = w.apply(obs)

x.shape, y.shape, wxy.shape, w.M

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(x, y, 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(x[mask], y[mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(x[mask], y[mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

In [0]:
wrr = np.array([[0, 0], [0, 5], [5, 0]])

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(wxy[..., 0], wxy[..., 1], 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(wxy[..., 0][mask], wxy[..., 1][mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(wxy[..., 0][mask], wxy[..., 1][mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

axs[1].plot(wrr[..., 0], wrr[..., 1], 'xr')

In [0]:
xy = np.matmul(np.linalg.inv(w.W), wxy.T).T + w.M
rr = np.matmul(np.linalg.inv(w.W), wrr.T).T + w.M

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(xy[..., 0], xy[..., 1], 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(xy[..., 0][mask], xy[..., 1][mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(xy[..., 0][mask], xy[..., 1][mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

axs[1].plot(rr[..., 0], rr[..., 1], 'xr')

In [0]:
# Fit profile to all stars within the color cut

king = King(transformation=w, R_max=6, bins=30)
king.histogram(obs)
[S_b, S_0, R_c, R_t], pcov = king.fit()

S_b, S_0, R_c, R_t

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=120)

R, log_S, log_S_sigma = king.get_log_S()

ax.errorbar(R, log_S, 3 * log_S_sigma)
ax.set_xlabel('deprojected R')
ax.set_ylabel(r'$\log \Sigma$')

R = np.linspace(0, 8, 200)
log_S = king.log_eval(R)
ax.plot(R, log_S)

In [0]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [0]:
# king.transformation.projection.pointing.posang

# king.transformation.Vh

np.arctan2(king.transformation.Vh[0, 1], king.transformation.Vh[0, 0])

In [0]:
ra0, dec0 = king.transformation.reverse(0, 0)
ra1, dec1 = king.transformation.reverse(0, 1)
print(ra, dec)

In [0]:
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.), dpi=240)
ax = f.add_subplot(projection=fov.projection.wcs)

fov.plot_catalog(ax, obs, c='gray', size=0.02, rasterized=True)

ell = king.get_ellipse(R_t)
fov.plot(ax, ell, native_frame='world', fmt='--r')

(S_b, S_0, R_c, R_t) = king.params
R = np.array([0, R_t, R_t, R_t, R_t])
phi = np.array([0, 0, np.pi / 2, np.pi, 3 / 2 * np.pi])

ra, dec = king.elliptic_to_world(R, phi, ctype='t')

c = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame='icrs')
ax.plot(c.ra.degree, c.dec.degree, 'x', transform=ax.get_transform('world'))

ax.invert_xaxis()
ax.grid()

In [0]:
obs.get_coords()[0].min(), obs.get_coords()[0].max()

In [0]:
obs.get_coords()[0].mean(), obs.get_coords()[1].mean()

In [0]:
gs = GridSpec(ncols=2, nrows=1, width_ratios=[3, 2])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = 2 * [None]

axs[0] = f.add_subplot(gs[0], projection=fov.projection.wcs)

(S_b, S_0, R_c, R_t) = king.params
xy = w.apply(obs)
r = np.sqrt(xy[:, 0]**2 + xy[:, 1]**2)

# mask = get_selection_mask(obs, observed=True, nb=True, blue=True, probcut=probcut)
# mask &= r > R_t 

mask = r > R_t

fov.plot_catalog(axs[0], obs, mask=mask, c=hex_to_rgb('#FF800E'), size=0.02, rasterized=True)
fov.plot_catalog(axs[0], obs, mask=~mask, c='gray', size=0.02, rasterized=True)

ell = king.get_ellipse(R_t, res=36)
fov.plot(axs[0], ell, fmt='--', native_frame='world', color=hex_to_rgb('#FF800E'), ls='--')

#ell = king.get_ellipse(R_c, res=36)
#fov.plot(axs[0], ell, fmt='-', color=hex_to_rgb('#FF800E'))

for pp in [(228.2, 67.5, 0),
           (226.3, 67.5, 0),
           (226.0, 66.9, 0),
           (228.1, 66.955, 40)]:
    # wfc = SubaruWFC(Pointing(227.3, 67.25))
    wfc = SubaruWFC(Pointing(*pp))
    pfi = SubaruPFI(wfc)
    #pfi.plot_field_of_view(axs[0], fov, blocks=True, c='b')
    pfi.plot_focal_plane(axs[0], fov, corners=True, c=hex_to_rgb('#006BA4'), ls='-')

axs[0].text(0.05, 0.9, 'a)', fontsize=8, transform=axs[0].transAxes)
axs[0].set_xlabel(r'right ascension')
axs[0].set_ylabel(r'declination')
#axs[0].yaxis.set_ticks(np.arange(16, 24.5, 1.0))
#axs[0].xaxis.set_ticks([0, 0.5, 1, 1.5])
#axs[0].set_xlim(-0.55, 2.4)
#axs[0].set_ylim(15.5, 24.5)
axs[0].invert_xaxis()
#axs[0].grid()

axs[0].tick_params(direction="in") #, top=True, right=True)
axs[0].grid()

####

axs[1] = f.add_subplot(gs[1])

cmd.plot_catalog(axs[1], obs, mask=mask, color='grey')
#cmd.plot_catalog(axs[1], obs, mask=mask, color='red')

#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

####

# axs[2] = f.add_subplot(gs[2])

# cmd.plot_catalog(axs[2], obs, mask=mask, color='grey')

####

axs[1].set_aspect('auto')

axs[1].text(0.075, 0.9, 'b)', fontsize=8, transform=axs[1].transAxes)
axs[1].set_xlabel(r'HSC $g - i$')
axs[1].set_ylabel(r'HSC $g$')

axs[1].yaxis.tick_right()
axs[1].yaxis.set_label_position("right")
axs[1].tick_params(direction="in", left=True, bottom=True, top=True, right=True)

f.tight_layout()

p0 = axs[0].get_position()
for ax in axs[1:]:
    p1 = ax.get_position()
    p1 = [p1.x0, p0.y0, p1.width, p0.height]
    ax.set_position(p1)

f.savefig('umi_hsc_tidal.pdf')

In [0]:
from pfs.ga.targeting.diagram import FP
from pfs.ga.targeting.allocation.algorithms import Brightness

In [0]:
gs = GridSpec(nrows=3, ncols=3, width_ratios=[2, 2, 4.3], height_ratios=[1, 1, 0.1])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

mask = get_selection_mask(obs, nb=True, blue=True, probcut=probcut)
targets = np.full_like(mask, False)
ll = []
axs = []
vmin = []
vmax = []

for pp, (si, sj) in zip(
        [(228.2, 67.5, 0),
         (226.3, 67.5, 0),
         (226.0, 66.9, 0),
         (228.1, 66.955, 40)],
        [(0, 0), (0, 1), (1, 0), (1, 1)]
):

    pointing = Pointing(*pp)
    wcs = WcsProjection(pointing, proj='TAN')
    wfc = SubaruWFC(pointing)
    pfi = SubaruPFI(wfc)

    fp = FP(projection=wfc)

    ax = f.add_subplot(gs[si, sj])
    pfi.plot_focal_plane(ax, fp, corners=True, color='b')

    radec = obs.get_coords()
    assoc = pfi.find_associations(radec, mask=mask)
    br = Brightness(cmd.axes[1].magnitude)
    sorting = br.argsort_assoc(obs, assoc, mask=mask)
    ix, ixm = assoc.pick_first(sorting)

    midx = tuple(a[ix[ixm]] for a in np.where(mask))    
    # Include selected targets in final list
    targets[midx] = True
    # Exclude selected targets from original list
    mask[midx] = False
    
    c = np.log10(assoc.counts())
    c[assoc.counts() == 0] = np.nan
    vmin.append(np.nanmin(c))
    vmax.append(c.max())
    l = pfi.plot_cobras(ax, fp, data=c, cmap=get_nice_cmap(from_black=True), vmax=2.0)
    ll.append(l)
    axs.append(ax)

for ax in axs:
    ax.set_xlim(-250, 250)
    ax.set_ylim(-250, 250)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    #ax.tick_params(direction="in", left=True, bottom=True, top=True, right=True)
    ax.tick_params(bottom=False, top=False, left=False, right=False, 
        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
    ax.set_xlabel(None)
    ax.set_ylabel(None)

ax = f.add_subplot(gs[2, :2])
cb = f.colorbar(ll[0], cax=ax, orientation='horizontal', extend='min')
cb.set_label('$\log_{10} \,\, N_\mathrm{target}$')
cb.ax.tick_params(color='k', direction='in', top=True, bottom=True)

ax = f.add_subplot(gs[:, 2])
cmd.plot_catalog(ax, obs, color='grey', rasterized=True)
cmd.plot_catalog(ax, obs, mask=targets, color='red', rasterized=True)

ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)
# ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
ax.set_xlabel(r'HSC $g - i$')
ax.set_ylabel(r'HSC $g$')
ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
#ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
ax.set_xlim(-0.85, 2.4)
ax.set_ylim(15.5, 24.5)
ax.invert_yaxis()
# ax.grid()
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.tick_params(direction="in", left=True, bottom=True, top=True, right=True)

f.tight_layout()

f.savefig('umi_hsc_target.pdf')

In [0]:
vmin, vmax

In [0]:
f, ax = plt.subplots(1, 1)

fp = FP(projection=wcs)

fp.plot_catalog(ax, obs)

In [0]:
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

wfc = SubaruWFC(pointing)
pfi = SubaruPFI(wfc)

#w = Whitening(projection=wcs)
#w.create(obs)

centers, radec = pfi.get_cobra_centers()

centers, radec

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

#l0 = fov.plot_catalog(ax, obs, size=0.1)
pfi.plot_focal_plane(ax)

ax.set_xlim(-200, 200)
ax.set_ylim(-200, 200)

## Fit foreground - member fraction

Here we fit the radial profile again but this time we only use stars that are within the fine color cuts and the coverage of the probability map. We keep the morphological parameters fixed.

In [0]:
mask = get_selection_mask(obs, nb=True, probcut=probcut)

In [0]:
plot_selection_mask(obs, mask)

In [0]:
king.histogram(obs, mask=mask)
[S_b, S_0, R_c, R_t], pcov = king.fit_nomorph()

# (185.89989551509436, 7239.798637878608, 0.8986542066851602, 4.581064733995127)
S_b, S_0, R_c, R_t

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=120)

R, log_S, log_S_sigma = king.get_log_S()

ax.errorbar(R, log_S, 3 * log_S_sigma)
ax.set_xlabel('deprojected R')
ax.set_ylabel(r'$\log \Sigma$')

R = np.linspace(0, 8, 200)
log_S = king.log_eval(R)
ax.plot(R, log_S)

# Look up membership probability

For each star, based on the CMD and elliptical radius

In [0]:
mask = get_selection_mask(obs, nb=True, probcut=probcut)

In [0]:
# Membership based on radial profile

R = king.R(obs)
S = king.eval(R, 0, S_0, R_c, R_t)
w_king = np.stack([S_b / (S + S_b), S / (S + S_b)], axis=-1)
lp_king = np.log(king.eval(R, 0, S_0, R_c, R_t) / S_b)

In [0]:
w_king.shape, mask.shape

In [0]:
plt.plot(R, lp_king, '.')

In [0]:
# Membership based on colors

lp_member, mask_member = pmap.lookup_lp_member(obs, mask=mask)
lp_member_king, mask_member_king = pmap.lookup_lp_member(obs, w_king[mask], mask=mask)

lp_member.shape, np.isnan(lp_member).sum(), np.isnan(lp_member[mask_member]).sum(), mask_member.shape, mask_member.sum()

In [0]:
plot_selection_mask(obs, mask=mask)

In [0]:
mask_member.shape, mask.shape, lp_member.shape

In [0]:
f, axs = plt.subplots(2, 2, figsize=(6, 6), dpi=120)

cmd.plot_catalog(axs[0, 0], obs, color=lp_member[..., 0][mask_member], mask=mask)
cmd.plot_catalog(axs[0, 1], obs, color=lp_member[..., 1][mask_member], mask=mask)

ccd.plot_catalog(axs[1, 0], obs, color=lp_member[..., 0][mask_member], mask=mask)
ccd.plot_catalog(axs[1, 1], obs, color=lp_member[..., 1][mask_member], mask=mask)

f.tight_layout()

In [0]:
f, axs = plt.subplots(2, 2, figsize=(6, 6), dpi=120)

cmd.plot_catalog(axs[0, 0], obs, color=lp_member_king[..., 0][mask_member_king], mask=mask)
cmd.plot_catalog(axs[0, 1], obs, color=lp_member_king[..., 1][mask_member_king], mask=mask)

ccd.plot_catalog(axs[1, 0], obs, color=lp_member_king[..., 0][mask_member_king], mask=mask)
ccd.plot_catalog(axs[1, 1], obs, color=lp_member_king[..., 1][mask_member_king], mask=mask)

f.tight_layout()

In [0]:
mask.shape, mask.sum(), lp_member.shape, mask_member.shape

In [0]:
f, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=120)

(S_b, S_0, R_c, R_t) = king.params
ell = king.get_ellipse(R_t)

l0 = fov.plot_catalog(axs[0], obs, mask=mask, size=0.5, color=lp_member[..., 1][mask_member], vmin=-2, vmax=0)
fov.plot(axs[0], ell, fmt='-r')
axs[0].set_title('CMD only')

l1 = fov.plot_catalog(axs[1], obs, mask=mask, size=0.5, color=lp_member_king[..., 1][mask_member_king], vmin=-2, vmax=0)
fov.plot(axs[1], ell, fmt='-r')
axs[1].set_title('CMD and radial profile')

for ax in axs:
    ax.set_xlim(-1.8, 1.8)
    ax.set_ylim(-1.8, 1.8)

f.colorbar(l0, ax=axs[0])
f.colorbar(l1, ax=axs[1])

# Everything in one function

In [0]:
from pfs.ga.targeting.allocation.algorithms import Brightness, Probability, Random

In [0]:
def plot_selection_mask(catalog, mask, fov=None, w=None, s=np.s_[:]):
    ii = 2
    if fov is not None: ii += 1
    if w is not None: ii += 1

    f, axs = plt.subplots(ii, 2, figsize=(6, 3 * ii), dpi=120)

    i = 0

    cmd.plot_catalog(axs[i, 0], catalog, mask=~mask, s=s, c='k')
    cmd.plot_catalog(axs[i, 0], catalog, mask=mask, s=s, c='r')
    cmd.plot_catalog(axs[i, 1], catalog, mask=mask, s=s, c='k')

    i += 1

    ccd.plot_catalog(axs[i, 0], catalog, mask=~mask, s=s, c='k')
    ccd.plot_catalog(axs[i, 0], catalog, mask=mask, s=s, c='r')
    ccd.plot_catalog(axs[i, 1], catalog, mask=mask, s=s, c='k')

    if fov is not None:

        i += 1

        fov.plot_catalog(axs[i, 0], catalog)
        fov.plot_catalog(axs[i, 0], catalog, mask=mask, color='r')
        fov.plot_catalog(axs[i, 1], catalog, mask=mask)

    if w is not None:

        i += 1

        wxy = w.apply(obs)

        axs[i, 0].plot(wxy[..., 0], wxy[..., 1], 'sk', markeredgecolor='None', ms=0.5)
        axs[i, 0].plot(wxy[..., 0][mask], wxy[..., 1][mask], 'sr', markeredgecolor='None', ms=0.5)
        axs[i, 0].set_aspect(1)

        axs[i, 1].plot(wxy[..., 0][mask], wxy[..., 1][mask], 'sk', markeredgecolor='None', ms=0.5)
        axs[i, 1].set_aspect(1)

    return f

In [0]:
def plot_radial_profile(catalog, mask, fov, profile, R):
    f, axs = plt.subplots(1, 2, figsize=(6, 2.5), dpi=120)

    profile.plot_histogram(axs[0])
    profile.plot_profile(axs[0])
    axs[0].axvline(R, c='r')

    axs[0].set_xlabel('deprojected R')
    axs[0].set_ylabel(r'$\log \Sigma$')

    fov.plot_catalog(axs[1], catalog, mask=mask)
    fov.plot_radial_profile(axs[1], profile, R)

    return f

In [0]:
def plot_probability_map(cmd, pmap):
    f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

    l0 = cmd.plot_probability_map(axs[0], pmap, 0)
    axs[0].set_title("non-member")
    
    l1 = cmd.plot_probability_map(axs[1], pmap, 1)
    axs[1].set_title("member")

    return f

In [0]:
def plot_probability_cmd(axs, cmd, ccd, obs, mask, lp_member, mask_member, s=None):
    axs[0, 0].set_title('All')
    axs[0, 1].set_title('Non-member probability')
    axs[0, 2].set_title('Member probability')

    cmd.plot_catalog(axs[0, 0], obs, color='k', mask=mask, s=s)
    cmd.plot_catalog(axs[0, 1], obs, color=lp_member[..., 0][mask_member], mask=mask, s=s)
    cmd.plot_catalog(axs[0, 2], obs, color=lp_member[..., 1][mask_member], mask=mask, s=s)

    ccd.plot_catalog(axs[1, 0], obs, color='k', mask=mask, s=s)
    ccd.plot_catalog(axs[1, 1], obs, color=lp_member[..., 0][mask_member], mask=mask, s=s)
    ccd.plot_catalog(axs[1, 2], obs, color=lp_member[..., 1][mask_member], mask=mask, s=s)

In [0]:
def plot_probability_fov(fov, obs, mask, lp_member, lp_member_king, mask_member, mask_member_king, king):
    f, axs = plt.subplots(1, 2, figsize=(8, 4), dpi=120)

    (S_b, S_0, R_c, R_t) = king.params
    ell1 = king.get_ellipse(R_t)
    ell2 = king.get_ellipse(R_c)

    l0 = fov.plot_catalog(axs[0], obs, mask=mask, size=0.5, color=lp_member[..., 1][mask_member], vmin=-2, vmax=0)
    fov.plot(axs[0], ell1, fmt='-b')
    fov.plot(axs[0], ell2, fmt='-r')
    axs[0].set_title('CMD only')

    l1 = fov.plot_catalog(axs[1], obs, mask=mask, size=0.5, color=lp_member_king[..., 1][mask_member_king], vmin=-2, vmax=0)
    fov.plot(axs[1], ell1, fmt='-b')
    fov.plot(axs[1], ell2, fmt='-r')
    axs[1].set_title('CMD and radial profile')

    for ax in axs:
        ax.set_xlim(-1.8, 1.8)
        ax.set_ylim(-1.8, 1.8)

    f.colorbar(l0, ax=axs[0])
    f.colorbar(l1, ax=axs[1])

    return f

In [0]:
def plot_assoc_hist(assoc):
    f, axs = plt.subplots(1, 2, figsize=(8, 3), dpi=120)

    counts = np.array(assoc.counts(), dtype=float)
    counts[counts == 0] = np.nan

    l = pfi.plot_fibers(axs[0], counts)
    f.colorbar(l, ax=axs[0])

    (S_b, S_0, R_c, R_t) = king.params
    ell = king.get_ellipse(0.3 * R_t, 500)
    fov.plot(axs[0], ell, mask_fov=False, fmt='-r')

    pfi.plot_focal_plane(axs[0])
    wfc.plot_focal_plane(axs[0])
    fov.apply(axs[0])
    axs[0].set_title("Number of stars per fiber")

    axs[0].set_xlim(-250, 250)
    axs[0].set_ylim(-250, 250)

    ####

    assoc.plot_histogram(axs[1])    

    return f

In [0]:
def plot_targeting(axs, assoc, ix, fiber_mask, lp_member, lp_member_king):
    lpp = 2 * [None]

    lpp[0] = (lp_member[..., 1][ix]) / np.log(10)
    lpp[1] = (lp_member_king[..., 1][ix]) / np.log(10)

    for i in range(len(lpp)):
        lpp[i][~fiber_mask] = np.nan    # Fibers with no targets

    fov = FOV(wfc)

    for ax, lp in zip(axs, lpp):
        l = pfi.plot_fibers(ax, lp, vmin=-2, vmax=0)

        (S_b, S_0, R_c, R_t) = king.params
        
        ell = king.get_ellipse(1/3 * R_t, 500)
        fov.plot(ax, ell, mask_fov=False, fmt='-b')
        ell = king.get_ellipse(2/3 * R_t, 500)
        fov.plot(ax, ell, mask_fov=False, fmt='-b')
        ell = king.get_ellipse(R_t, 500)
        fov.plot(ax, ell, mask_fov=False, fmt='-b')
        ell = king.get_ellipse(R_c, 500)
        fov.plot(ax, ell, mask_fov=False, fmt='-r')
        
        pfi.plot_focal_plane(ax)
        wfc.plot_focal_plane(ax)
        fov.apply(ax)

        ax.set_xlim(-250, 250)
        ax.set_ylim(-250, 250)
        
        f.colorbar(l, ax=ax, orientation='horizontal')

    axs[0].set_title('CMD-based probability')
    axs[1].set_title('Success rate')

In [0]:
def plot_targeting_hist(ax, wr, ix, fiber_mask, lp_member, lp_member_king):
    nan_mask = np.isnan(lp_member[..., 1][ix[fiber_mask]])

    bins = np.linspace(0, 1.2, 31)

    hist, _ = np.histogram(wr[ix[fiber_mask]][~nan_mask], bins=bins)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='all targets')

    w = np.exp(lp_member[..., 1][ix[fiber_mask]][~nan_mask])
    hist, _ = np.histogram(wr[ix[fiber_mask]][~nan_mask], weights=w, bins=bins)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='color only')

    w = np.exp(lp_member_king[..., 1][ix[fiber_mask]][~nan_mask])
    hist, _ = np.histogram(wr[ix[fiber_mask]][~nan_mask], weights=w, bins=bins)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='color + radial profile')

    ax.set_title('Radial distribution')
    ax.set_ylim(-5, 110)
    ax.set_xlabel('elliptical radius R / R_t')
    ax.set_ylabel('counts')

def plot_brightness_hist(ax, m, ix, fiber_mask, lp_member, lp_member_king):
    nan_mask = np.isnan(lp_member[..., 1][ix[fiber_mask]])

    hist, bins = np.histogram(m[ix[fiber_mask]][~nan_mask], bins=30)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='targeted stars')

    w = np.exp(lp_member[..., 1][ix[fiber_mask]][~nan_mask])
    hist, bins = np.histogram(m[ix[fiber_mask]][~nan_mask], weights=w, bins=30)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='w = p_mem_CMD')

    w = np.exp(lp_member_king[..., 1][ix[fiber_mask]][~nan_mask])
    hist, bins = np.histogram(m[ix[fiber_mask]][~nan_mask], weights=w, bins=30)
    ax.step(0.5 * (bins[1:] + bins[:-1]), hist, label='w = p_mem_CMD * p_mem_rad')

    ax.legend()
    
    ax.set_title('Magnitude distribution')
    ax.set_ylim(-5, 250)
    ax.set_xlabel('HSC g')
    ax.set_ylabel('counts')

In [0]:
# obs
# sim
# cmd
nb = True
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
plot_selection = True
plot_profile = True
plot_pmap = True
plot_probability_scatter = True
plot_associations = True

wcs = WcsProjection(pointing, proj='TAN')
fov = FOV(projection=wcs)

wfc = SubaruWFC(pointing)
pfi = SubaruPFI(wfc)

w = Whitening(projection=wcs)
w.create(obs)

# Fit the radial profile to all data in order to pin morphological parameters
king = King(transformation=w, R_max=6, bins=30)
king.histogram(obs)
[S_b, S_0, R_c, R_t], pcov = king.fit()

if plot_profile:
    f = plot_radial_profile(obs, None, fov, king, R_t)
    f.suptitle('Radial profile without color cuts, NB={}'.format(nb))
    f.tight_layout()

# To create the first iteration of the probability map, take everything within the coarse
# color cuts. This will be used to determine the probability map contour
mask = get_selection_mask(sim, nb=False, probcut=None)

if plot_selection:
    f = plot_selection_mask(sim, sim.apply_categories(mask), fov=None, w=None, s=np.s_[:40000])
    f.suptitle('Simulation with coarse color selection, NB={}'.format(nb))
    f.tight_layout()

pmap = ProbabilityMap(cmd.axes)
pmap.from_simulation(sim, bins=[100, 100], merge_list=[np.s_[:10], np.s_[10:]], mask=mask)
pmap.maximum_filter()

if plot_pmap:
    f = plot_probability_map(cmd, pmap)
    f.suptitle('Probablity map with coarse color selections, NB={}'.format(False))
    f.tight_layout()

# This will be the probability map outline for further selections
probcut = ProbabilityCut(pmap, 1, np.log(0.0001))

# Fine color selection (including NB)
mask = get_selection_mask(sim, nb=nb, probcut=probcut)

# Recreate probability map with NB color
pmap = ProbabilityMap(cmd.axes)
pmap.from_simulation(sim, bins=[100, 100], merge_list=[np.s_[:10], np.s_[10:]], mask=mask)
pmap.maximum_filter()

# This will be the probability map outline for further selections
probcut = ProbabilityCut(pmap, 1, np.log(0.0001))

if plot_pmap:
    f = plot_probability_map(cmd, pmap)
    f.suptitle('Probablity map with fine color selections, NB={}'.format(nb))
    f.tight_layout()

if plot_selection:
    f = plot_selection_mask(sim, sim.apply_categories(mask), fov=None, w=None, s=np.s_[:40000])
    f.suptitle('Simulation with fine color selection, NB={}'.format(nb))
    f.tight_layout()

mask = get_selection_mask(obs, nb=nb, probcut=probcut)

if plot_selection:
    f = plot_selection_mask(obs, mask, fov=fov, w=w, s=np.s_[:40000])
    f.suptitle('Observations with fine color selection, NB={}'.format(nb))
    f.tight_layout()

# Fit the radial profile again to determine the foreground/member fraction after selection
# but keep morphological parameters constant
king.histogram(obs, mask=mask)
[S_b, S_0, R_c, R_t], pcov = king.fit_nomorph()

if plot_profile:
    f = plot_radial_profile(obs, mask, fov, king, R_t)
    f.suptitle('Radial profile with fine color cuts, NB={}'.format(nb))
    f.tight_layout()

# Membership based on radial profile
R = king.R(obs)
S = king.eval(R, 0, S_0, R_c, R_t)
w_king = np.stack([S_b / (S + S_b), S / (S + S_b)], axis=-1)
lp_king = np.log(king.eval(R, 0, S_0, R_c, R_t) / S_b)

# Membership based on colors
lp_member, mask_member = pmap.lookup_lp_member(obs, mask=mask)
lp_member_king, mask_member_king = pmap.lookup_lp_member(obs, w_king[mask], mask=mask)

if plot_probability_scatter:
    f, axs = plt.subplots(2, 3, figsize=(6, 4), dpi=120)
    plot_probability_cmd(axs, cmd, ccd, obs, mask, lp_member, mask_member)
    f.suptitle('CMD-based probability, NB={}'.format(nb))

    f, axs = plt.subplots(2, 3, figsize=(6, 4), dpi=120)
    plot_probability_cmd(axs, cmd, ccd, obs, mask, lp_member_king, mask_member_king)
    f.suptitle('CMD and radial-profile-based probability, NB={}'.format(nb))

    f = plot_probability_fov(fov, obs, mask, lp_member, lp_member_king, mask_member, mask_member_king, king)
    f.suptitle('CMD and radial-profile-based probability, NB={}'.format(nb))

# Find fiber associations for the pointing, this is still independent on
# the actual targeting algorithms
assoc = pfi.find_associations(obs, mask=mask)

if plot_associations:
    f = plot_assoc_hist(assoc)
    f.suptitle('CMD and radial-profile-based probability, NB={}'.format(nb))
    f.tight_layout()

In [0]:
lp_member.shape, mask.sum()

In [0]:
from matplotlib.gridspec import GridSpec

In [0]:
# Run the various targeting algorithms
for tt in range(5):
    print(tt)

    if tt == 0:
        suptitle = 'pick stars randomly, NB={}'.format(nb)
        ra = Random()
        ss = ra.argsort_assoc(assoc)
    elif tt == 1:
        suptitle = 'pick faintest stars, NB={}'.format(nb)
        ba = Brightness(cmd.axes[1].magnitude)
        ss = ba.argsort_assoc(obs, assoc, mask=mask, reverse=True)
    elif tt == 2:
        suptitle = 'pick brightest stars, NB={}'.format(nb)
        ba = Brightness(cmd.axes[1].magnitude)
        ss = ba.argsort_assoc(obs, assoc, mask=mask)
    elif tt == 3:
        suptitle = 'pick most probable member stars, NB={}'.format(nb)
        pa = Probability()
        ss = pa.argsort_assoc(lp_member[..., 1], assoc, reverse=True)
    elif tt == 4:
        suptitle = 'pick least probable member stars, NB={}'.format(nb)
        pa = Probability()
        ss = pa.argsort_assoc(lp_member[..., 1], assoc)
    else:
        raise NotImplementedError()

        
    gs = GridSpec(6, 3)
    f = plt.figure(figsize=(12, 9), dpi=120)

    axs = [f.add_subplot(gs[0:4, 0]),
           f.add_subplot(gs[0:4, 1])]
    ix, fiber_mask = assoc.pick_first(sorting=ss)
    plot_targeting(axs, assoc, ix, fiber_mask, lp_member, lp_member_king)

    # axs = np.array([[f.add_subplot(gs[0, 4]),
    #                  f.add_subplot(gs[0, 5])],
    #                 [f.add_subplot(gs[1, 4]),
    #                  f.add_subplot(gs[1, 5])],
    #                 [f.add_subplot(gs[2, 4]),
    #                  f.add_subplot(gs[2, 5])]])
    # plot_probability_cmd(axs.T, cmd, ccd, obs, mask, lp_member, mask_member, s=ix)

    ax = f.add_subplot(gs[0:3, 2])
    cmd.plot_catalog(ax, obs, color='gray', alpha=0.3)
    cmd.plot_catalog(ax, obs, color='r', size=1, mask=mask, s=ix)

    ax = f.add_subplot(gs[3:6, 2])
    ccd.plot_catalog(ax, obs, color='gray', alpha=0.3)
    ccd.plot_catalog(ax, obs, color='r', size=1, mask=mask, s=ix)

    ax = f.add_subplot(gs[4:6, 0])
    wr = king.R(obs) / R_t
    plot_targeting_hist(ax, wr[mask], ix, fiber_mask, lp_member, lp_member_king)

    ax = f.add_subplot(gs[4:6, 1])
    m = obs.get_magnitude(cmd.axes[1].magnitude)[mask]
    plot_brightness_hist(ax, m, ix, fiber_mask, lp_member, lp_member_king)

#     axs[1].legend(loc='upper right')

    f.suptitle(suptitle)
    f.tight_layout()